# Введение в обработку естественного языка
## Урок 12. Модель Transformer-2
### Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
#### 1. Проверить насколько хорошо она суммаризирует

In [11]:
# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [3]:
test_records = read_gazeta_records("gazeta_test.txt")

In [4]:
import pandas as pd

dataset = pd.DataFrame(test_records)

In [5]:
dataset = dataset.drop(['url', 'date'], axis=1)

In [6]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")
model = MBartForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

In [7]:
dataset = dataset[:round(len(dataset)/6)]
len(dataset)

962

In [17]:
data = dataset['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

In [18]:
dataset['pred_summary'] = summaries
dataset.sample(5)

,text,title,summary,pred_summary
549,Президент России Владимир Путин подписал закон...,Вчетверо ниже средней: в России повысили миним...,В России с 1 января 2020 года вводится новая в...,Президент России Владимир Путин подписал закон...
211,Украине указали на ее место в споре по российс...,Оставим без денег: в Германии предупредили Укр...,Украина может остаться без финансовой помощи Е...,Украина может остаться без финансовой поддержк...
546,Нынешний сезон пока складывается провально не ...,Вторая попытка: Чалова переманивают в Англию,Нападающий московского ЦСКА Федор Чалов может ...,Нападающий московского ЦСКА Федор Чалов может ...
617,Лидером рейтинга жена бывшего мэра Москвы Юрия...,Рекордное состояние: самые богатые женщины России,Журнал Forbes составил рейтинг 25 самых богаты...,Лидером рейтинга богатейших людей России по ве...
489,Новейшие истребители F-35 Lightning II ВВС США...,«Удар ногой в дверь»: Вашингтон похоронил ДРСМД,Американские истребители пятого поколения вско...,Американские военные «научились» передавать са...


#### 2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [8]:
from transformers import AutoTokenizer, EncoderDecoderModel

model_name = "IlyaGusev/rubert_telegram_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
model = EncoderDecoderModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/76.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [9]:
dataset = dataset[:round(len(dataset)/6)]
len(dataset)

160

In [10]:
data = dataset['text']

headlines = []

for row in data:
    input_ids = tokenizer(
        [row],
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=64,
        no_repeat_ngram_size=3,
        num_beams=10,
        top_p=0.95
    )[0]
    
    headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    headlines.append(headline)

dataset['pred_headline'] = headlines
dataset.sample(5)

,text,title,summary,pred_headline
19,Глава Роскомнадзора Александр Жаров объявил о ...,«Мы умеем ждать»: как Роскомнадзор побеждает G...,Глава Роскомнадзора Александр Жаров заявил о т...,Google увеличил уровень фильтрации контента в ...
116,В Токио прошел митинг за возвращение «северных...,«Верните северные территории»: японцы вышли на...,Около 500 человек вышли на митинг в Токио за в...,В Японии прошел митинг за возвращение « северн...
8,Российский актер и театральный педагог Виталий...,Умер актер сериала «Улицы разбитых фонарей»,Актер сериала «Улицы разбитых фонарей» Виталий...,Умер российский актер и театральный педагог Ви...
53,Пока в Европе занимаются развитием возобновляе...,Не до экологии: почему Китай ставит на уголь,Мировой лидер по выбросам углекислого газа — К...,Китай готовится построить новые угольных станции
103,Накануне старта чемпионата мира по биатлону в ...,«У нее маразм»: Фуркад сцепился с российской ч...,Французский биатлонист Мартен Фуркад поругался...,«Не пойман — не ворует». Мать биатлонистки Рез...


In [12]:
from datasets import load_metric

rouge_score = load_metric("rouge")

predictions = dataset['pred_headline']
references = dataset['title']

head_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
head_scores

{'rouge1': AggregateScore(low=Score(precision=0.2727272727272727, recall=0.6666666666666666, fmeasure=0.3870967741935484), mid=Score(precision=0.2727272727272727, recall=0.6666666666666666, fmeasure=0.3870967741935484), high=Score(precision=0.2727272727272727, recall=0.6666666666666666, fmeasure=0.3870967741935484)),
 'rouge2': AggregateScore(low=Score(precision=0.05747126436781609, recall=0.14285714285714285, fmeasure=0.08196721311475409), mid=Score(precision=0.05747126436781609, recall=0.14285714285714285, fmeasure=0.08196721311475409), high=Score(precision=0.05747126436781609, recall=0.14285714285714285, fmeasure=0.08196721311475409)),
 'rougeL': AggregateScore(low=Score(precision=0.23863636363636365, recall=0.5833333333333334, fmeasure=0.33870967741935487), mid=Score(precision=0.23863636363636365, recall=0.5833333333333334, fmeasure=0.33870967741935487), high=Score(precision=0.23863636363636365, recall=0.5833333333333334, fmeasure=0.33870967741935487)),
 'rougeLsum': AggregateScore